## Team

Saurabh Bodas

Ryan Harty

David Owen

Pooja Shah

Emily Wilkins

# Task A

In [1]:
# Don't run anything but the marked cells in Task A to avoid running the whole scraper

!pip install selenium
# !apt-get -q update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

You should consider upgrading via the 'python -m pip install --upgrade pip' command.
'apt' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
from google.colab import drive 
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [14]:
# run this even if not scraping
import pandas as pd
from pandas import Series, DataFrame
from time import sleep

In [ ]:
df = DataFrame(columns=['product_name','product_review','user_rating'])
driver = webdriver.Chrome('chromedriver', options=chrome_options)
i = 1
while i < 250+1:
    i += 1
    driver.get('https://www.beeradvocate.com/beer/top-rated/')
    beer_name = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/table/tbody/tr['+str(i)+']/td[2]/a/b')[0].text
    beer_link = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/table/tbody/tr['+str(i)+']/td[2]/a')[0].get_attribute('href')
  
    driver.get(beer_link)
    j = 0
    while j < 25:
        j += 1
        try:
            rating = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div['+str(j)+']/div[2]/span[1]')[0].text
            review = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div['+str(j)+']/div[2]')[0].text.split('\n')[3:-4]
            review = ' '.join(review)
            df = df.append({'product_name':beer_name,'product_review':review,'user_rating':rating}, ignore_index = True)
        except IndexError:
            print('Beer',i-1,'has less than 25 reviews. Error on review',j)
            print(sys.exc_info()[0])
            break
    sleep(.5)
    if (i%50 == 0):
        print('On beer',i-1)

driver.quit()

In [ ]:
df.to_csv('hw2_scraped.csv', index = False)

# Task B

Steps for making spaCy work with Jupyter Notebook:

create a virtual environment: conda create -n textclass python=3.7 anaconda

activate it: conda activate textclass

make sure jupyter is installed: conda install -n textclass jupyter

install spacy: conda install -n textclass spacy

click 'y' if it asks to downgrade

run jupyter notebook inside the venv and it should work

(can repeat install with nltk inside venv if it doesn't work)

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
sw = set(nltk.corpus.stopwords.words('english'))
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\drs75\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\drs75\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
df = pd.read_csv('hw2_scraped.csv')
df = df[df['product_review'].notnull()]
df.head()

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,"2016 Silver Wax. Aroma has whiskey, maple, tof...",4.80
1,Kentucky Brunch Brand Stout,The beer pours Pitch Black with a frothy tan h...,4.74
2,Kentucky Brunch Brand Stout,Probably the smoothest beer I have ever had. S...,4.68
3,Kentucky Brunch Brand Stout,"Dark black, very thick, a little bit of tan he...",5.00
4,Kentucky Brunch Brand Stout,Poured black as ink with thin ruby edges at 58...,4.97


In [75]:
word_list

0       [2016, Silver, Wax, ., Aroma, has, whiskey, ,,...
1       [The, beer, pours, Pitch, Black, with, a, frot...
2       [Probably, the, smoothest, beer, I, have, ever...
3       [Dark, black, ,, very, thick, ,, a, little, bi...
4       [Poured, black, as, ink, with, thin, ruby, edg...
                              ...                        
6205    [Bottle, poured, ;, very, little, head, and, c...
6206    [First, review, of, Madagascar, :, 2017, vinta...
6207    [Yeah, ,, this, works, for, me, !, Lots, of, v...
6208    [Reviewing, from, notes, ., Thanks, Brandon, f...
6209    [2015, vintage, bottle, Tart, cherry, on, the,...
Name: product_review, Length: 6206, dtype: object

In [ ]:
word_list

In [77]:
# first, look for which of the specified attributes appear the most
# we will use the top 3
review_tokenized = df['product_review'].apply(nltk.word_tokenize)
word_list = []
for words in review_tokenized: # converting list of lists into one long list of words, dropping numbers and stopwords
    word_list += [w.lower() for w in words if len(w) > 1 and not w.isnumeric() and w.lower() not in sw]

In [78]:
fdist = nltk.FreqDist(word_list)
attr = ['aggressive','balanced','complex','crisp','fruity','hoppy','malty','robust']

attr_fdist = Series({i:fdist[i] for i in attr if i in fdist}).sort_values(ascending = False)
attr_fdist

balanced      843
complex       611
fruity        498
crisp         312
hoppy         273
malty         186
robust        113
aggressive     83
dtype: int64

So, the three attributes we will pick are "complex", "fruity", and "crisp". We have excluded "balanced" because it is defined as a mix of sweet and bitter and may conflict with sentiments for "fruity", as well as not making much sense for someone to want balance *and* fruitiness.

In [33]:
# create input file of our top three attributes
attr_file = open('attributes.txt','w')
for i in range(1,4):
    attr_file.write(attr_fdist.index.values[i] + '\n')
attr_file.close()

# Task C

In [22]:
import spacy
# run to get the actual word vectors after pip installing 'spacy'
# probably don't need to run this more than once
# remove the ! and put this in the command terminal; it's easier IMO
#!python -m spacy download en_core_web_lg (or md if large doesn't provide a big bonus)
import en_core_web_lg
nlp = en_core_web_lg.load()

In [43]:
attr_infile = open('attributes.txt','r')
attr_words = attr_infile.read().split() # if attributes are spaced out or on separate lines, this will read them correctly
attr_use = ' '.join(attr_words) # converts to spacing for nlp (in case the infile wasn't that way already)
attr_infile.close()

In [79]:
# this takes forever
df_sim = df[['product_name','product_review']] # copying over from the original df
attr_token = nlp(attr_use)
tokens = df['product_review'].apply(nlp)

df_sim['similarity_score'] = tokens.apply(attr_token.similarity) # adding similarity score row

In [84]:
df_sim_300 = df_sim.sort_values(by = 'similarity_score', ascending = False)[:300]
df_sim_300.to_csv('similarity.csv', index = False)
df_sim_300.head()

,product_name,product_review,similarity_score
4105,Saison Bernice,Tasted from bottle. Classic farmhouse saison. ...,0.793729
732,Duck Duck Gooze,On tap at Beavertown Extravaganza. Aroma has s...,0.770780
4750,Gueuze 100% Lambic,375ml bottled 17 May 2016 Pours cloudy hazy a...,0.766651
1111,Double Galaxy,"Huge notes of dank tropical citrus, pineapple,...",0.766617
3009,Pseudo Sue,Murky creamsicle orange in color. Poured with ...,0.764095


# Task D

In [2]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer() #initialising the VADER sentiment analyser

In [21]:
def sentiment_scores(review): 
    score = analyser.polarity_scores(review) #the polarity_scores function gives the scores for each review
    return score['compound'] #VADER gives us the negative, neutral, positive and compound scores - we only care about the compound one

In [16]:
reviews_300 = pd.read_csv('similarity.csv')
reviews_300.head()

,product_name,product_review,similarity_score
0,Saison Bernice,Tasted from bottle. Classic farmhouse saison. ...,0.793729
1,Duck Duck Gooze,On tap at Beavertown Extravaganza. Aroma has s...,0.770780
2,Gueuze 100% Lambic,375ml bottled 17 May 2016 Pours cloudy hazy a...,0.766651
3,Double Galaxy,"Huge notes of dank tropical citrus, pineapple,...",0.766617
4,Pseudo Sue,Murky creamsicle orange in color. Poured with ...,0.764095


In [25]:
reviews_300['sentiment_score'] = reviews_300['product_review'].apply(sentiment_scores) #finding the sentiment scores of each product review
reviews_300 = reviews_300.sort_values(by = 'sentiment_score', ascending = False)
reviews_300.head()

,product_name,product_review,similarity_score,sentiment_score
150,Barrel-Aged Sump Coffee Stout,Pitch black and nearly opaque with dark chocol...,0.714936,0.9967
87,Ghost In The Machine,"12 ounce bottle into tulip glass, bottled on 8...",0.724401,0.9966
78,Emerald Grouper,"16 ounce can into tulip glass, canned on 9/24/...",0.725506,0.9961
169,Double Dry Hopped Congress Street,"16 ounce can into tulip glass, canned on 4/11/...",0.713833,0.9958
140,Art,"375ml bottle, Batch 5, dated March 18, 2015, p...",0.715951,0.9951


In [50]:
average_scores = reviews_300.groupby('product_name').mean()

In [51]:
average_scores = average_scores.sort_values(by = 'sentiment_score', ascending = False)
average_scores.head()

,similarity_score,sentiment_score
product_name,,
Barrel-Aged Sump Coffee Stout,0.714936,0.9967
Expedition Stout - Bourbon Barrel-Aged,0.706004,0.9818
Assassin,0.715713,0.9817
Speedway Stout - Vietnamese Coffee,0.748050,0.9813
Alter Ego,0.714305,0.9776


In [52]:
reviews_300.to_csv('sentiment.csv', index = False)

In [38]:
average_scores.to_csv('average_scores.csv')

# Task E

Based on the above similarity and sentiment scores, we would recommend the following 3 beers to the customer:
    1. Barrel-Agd Sump Coffee Stout
    2. Expedition Stout - Bourbon Barrel-Aged
    3. Assassin

The average similarity and sentiment scores for these beers are as follows:

In [53]:
average_scores.head(3)

,similarity_score,sentiment_score
product_name,,
Barrel-Aged Sump Coffee Stout,0.714936,0.9967
Expedition Stout - Bourbon Barrel-Aged,0.706004,0.9818
Assassin,0.715713,0.9817


# Task F

In [54]:
total_average_ratings = df.groupby('product_name').mean().sort_values(by = 'user_rating', ascending = False)

Purely based on the user ratings of the entire dataset, we would recommend the following 3 beers to the user:

In [55]:
total_average_ratings.head(3)

,user_rating
product_name,
Kentucky Brunch Brand Stout,4.812000
Chemtrailmix,4.811176
Barrel-Aged Abraxas,4.796400


The similarity and sentiment scores for these beers with the user preferences are:

In [56]:
beer_by_ratings = total_average_ratings.head(3).index
average_scores.loc[beer_by_ratings]

C:\Users\drs75\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


,similarity_score,sentiment_score
product_name,,
Kentucky Brunch Brand Stout,NaN,NaN
Chemtrailmix,NaN,NaN
Barrel-Aged Abraxas,0.71231,0.8979
